<br><br><br>
<span style="color:red;font-size:60px">Spark SQL</span>
<p>
    <br>


<li>Apache Spark module for <span style="color:red">structured data processing</span></li>
<li>Implemented through two data structures</li>
<ul>
    <li><b>Datasets</b>: A distributed collection of data. Datasets are based on RDDs but are strongly typed and contain a optimized execution engine for fast execution. Available only in Scala and Java (mainly because Python is untyped)</li>
    <li><b>Dataframes</b>: a dataset organized into named columns. Available in all all APIs.It is untyped.</li>
    <li>The key difference between the two is that datasets are typed while dataframes are untyped</li>
</ul>
<li>Spark SQL uses DataFrames to provide structure to a data object</li>
        <ul>
            <li>A spark dataframe is a table like structure </li>
            <li>Data is organized in <span style="color:blue">named columns</span></li>
   <li>dataframes are distributed across nodes in a cluster</li>
        <li> dataframes are in-memory data objects</li>
    <li>dataframes support lazy evaluation (transformations and actions)</li>
        </ul>
<li>Spark SQL provides SQL syntax to query dataframes</li>

<li>Dataframes and SQL queries are optimized for execution speed</li>


<br><br>
<span style="color:green;font-size:xx-large">Spark Session</span>
<p>
    <li>The entry point for a spark application</li>
    <li>Early versions of Spark were RDD based and had only a Spark Context</li>
    <li>Later, support was added for SQL in an SQLContext and for Hive (a data warehousing framework) in a HiveContext</li>
    <li>SparkSession combines all these into a single global context that contains support for RDDs (spark context) and SQL (Sql context)</li>
    <li>A spark session automatically creates a spark context. Jupyter notebooks don't need to create either</li>
    <li>set config options (in the example below, we're setting the size of a partition in MB when shuffling data</li>
    

In [1]:
import org.apache.spark.sql.SparkSession
val sparkSession = SparkSession
                    .builder()
                    .appName("myApp")
                    .config("spark.sql.shuffle.partitions", 500)
                    .getOrCreate()

Intitializing Scala interpreter ...

Spark Web UI available at http://10.56.168.254:4041
SparkContext available as 'sc' (version = 3.3.0, master = local[*], app id = local-1667759003977)
SparkSession available as 'spark'


22/11/06 13:23:27 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


import org.apache.spark.sql.SparkSession
sparkSession: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@6200ca28


In [2]:
sparkSession.sparkContext

res0: org.apache.spark.SparkContext = org.apache.spark.SparkContext@1526c7d4


In [3]:
sc

res1: org.apache.spark.SparkContext = org.apache.spark.SparkContext@1526c7d4


In [4]:
spark

res2: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@6200ca28


In [5]:
sparkSession

res3: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@6200ca28


<br><br>
<span style="color:green;font-size:xx-large">Creating dataframes</span>
<p>
<li>From a Scala sequence</li>
<li>From an RDD</li>
<li>Reading data from a file (csv, json, other formats)</li>
<li>Scala Spark dataframes are composed of Row objects</li>

<br><br>
<span style="color:blue;font-size:x-large">From a scala sequence or an RDD</span>
<li><span style="color:blue">toDF</span> converts a scala sequence or an rdd into a data frame</li>


In [6]:
val data = Seq(("Megan",200000),("Gao",450000),("Antonio",120000))//ordered,immutable collection
val rdd = sc.parallelize(data)
val df_from_data = data.toDF("Name","Income")
val df_from_rdd = rdd.toDF("Name","Income")

//step 1,2, and 4. actually only have one step, 
//because rdd is not initiated, and action will determine what exactly transformation useful.

data: Seq[(String, Int)] = List((Megan,200000), (Gao,450000), (Antonio,120000))
rdd: org.apache.spark.rdd.RDD[(String, Int)] = ParallelCollectionRDD[0] at parallelize at <console>:26
df_from_data: org.apache.spark.sql.DataFrame = [Name: string, Income: int]
df_from_rdd: org.apache.spark.sql.DataFrame = [Name: string, Income: int]


In [7]:
df_from_data.printSchema
df_from_rdd.printSchema

root
 |-- Name: string (nullable = true)
 |-- Income: integer (nullable = false)

root
 |-- Name: string (nullable = true)
 |-- Income: integer (nullable = false)



<br><br>
<span style="color:blue;font-size:x-large">from a csv file</span>
<p>
    <div class="list">
<li>The spark function <span style="color:blue">read</span> reads the contents of a file into a dataframe</li>
<li>The .format option specifies the format (csv, json, user-defined format)</li>
        <li>Specify a header if the csv file has a (single) header line</li>
        <li>Specify "inferschema" if you want spark to figure out the data types</li>


In [8]:
!cat employees.csv

name,salary
Michael,3000
Andy,4500
Justin,3500
Berta,4000



In [6]:
val df = spark.read //Jupyter automatically creates a sparksession with the identifer "spark"
    .format("csv")
    .option("header","true")
    .option("inferschema","true")//"Michael" is a string, 3000 is an interger
   .csv("employees.csv")
df.printSchema

root
 |-- name: string (nullable = true)
 |-- salary: integer (nullable = true)



df: org.apache.spark.sql.DataFrame = [name: string, salary: int]


In [7]:
df.schema

res5: org.apache.spark.sql.types.StructType = StructType(StructField(name,StringType,true),StructField(salary,IntegerType,true))


<br><br>
<span style="color:blue;font-size:x-large">From a json file</span>
<p>
<li><span style="color:blue">read.json</span> reads json files</li>
<li>JSON: Must be single line json files</li>
<li>Each record must be in a single line with no line breaks</li>



<span style="font-size:20px">
<b>Single line json (This is OK!)</b>
    <p>
{"name":"Michael", "salary":3000,"positions":{"NYC":1,"Palo Alto":2}}<br>
{"name":"Andy", "salary":4500,"positions":{"NYC":1,"Palo Alto":2}}<br>
{"name":"Justin", "salary":3500,"positions":{"NYC":1,"Palo Alto":2}}<br>
{"name":"Berta", "salary":4000,"positions":{"NYC":1,"Palo Alto":2}}<br>
    
    
<b>Multi-line json (This is NOT OK!)</b>
        <p>
{"name":"Michael", "salary":3000,"positions":{<br>
    "NYC":1,<br>
    "Palo Alto":2}}<br>
{"name":"Andy", "salary":4500,"positions":{<br>
    "NYC":1,<br>
    "Palo Alto":2}}<br>
{"name":"Justin", "salary":3500,"positions":{<br>
    "NYC":1,<br>
    "Palo Alto":2}}<br>
{"name":"Berta", "salary":4000,"positions":{<br>
    "NYC":1,<br>
    "Palo Alto":2}}<br>
    </p>
    </span>

In [10]:
!cat employees_singleline.json

{"name":"Michael", "salary":3000,"positions":{"NYC":1,"Palo Alto":2}}
{"name":"Andy", "salary":4500,"positions":{"NYC":1}}
{"name":"Justin", "salary":3500,"positions":{"NYC":1,"Palo Alto":2}}
{"name":"Berta", "salary":4000,"positions":{"NYC":1,"Palo Alto":2}}



In [11]:
val df = spark.read.json("employees_singleline.json")
df.printSchema

root
 |-- name: string (nullable = true)
 |-- positions: struct (nullable = true)
 |    |-- NYC: long (nullable = true)
 |    |-- Palo Alto: long (nullable = true)
 |-- salary: long (nullable = true)



df: org.apache.spark.sql.DataFrame = [name: string, positions: struct<NYC: bigint, Palo Alto: bigint> ... 1 more field]


In [17]:
df.schema

res10: org.apache.spark.sql.types.StructType = StructType(StructField(name,StringType,true),StructField(positions,StructType(StructField(NYC,LongType,true),StructField(Palo Alto,LongType,true)),true),StructField(salary,LongType,true))


In [13]:
df.show

+-------+---------+------+
|   name|positions|salary|
+-------+---------+------+
|Michael|   {1, 2}|  3000|
|   Andy|{1, null}|  4500|
| Justin|   {1, 2}|  3500|
|  Berta|   {1, 2}|  4000|
+-------+---------+------+



<br><br>
<span style="color:blue;font-size:x-large">Programmatically specifying the schema</span>
<li>Often, data arrives in a Spark application without a mechanism to infer the schema</li>
<li>Since each df row is in a Spark Row object, it is possible to construct the schema programmatically</li>

In [1]:
val data = Array("John,33,245233.234","Jill,23,132987.22","Qing,54,782344.22","Rahul,50,389223,54")

Intitializing Scala interpreter ...

Spark Web UI available at http://10.56.160.213:4046
SparkContext available as 'sc' (version = 3.3.0, master = local[*], app id = local-1671153413116)
SparkSession available as 'spark'


data: Array[String] = Array(John,33,245233.234, Jill,23,132987.22, Qing,54,782344.22, Rahul,50,389223,54)


- Python float
- Pandas float64

<br><br>
<span style="color:blue;font-size:large">Construct a schema</span>
<p>
        <li>import SPARK SQL types (StringType, IntegerType, etc.)</li>
        <li>import SPARK SQL Row object</li>
        <li>construct a StructField type for each column</li>
        <li>construct a StructType for the schema</li>

In [9]:
import org.apache.spark.sql.types._ //spark dataframe type
import org.apache.spark.sql.Row //put f1,f2,f3 as row type
//for each column
val f1 = StructField("name",StringType,nullable=false)
val f2 = StructField("age",IntegerType,nullable=true)
val f3 = StructField("income",DoubleType,nullable=true)

val schema = StructType(Array(f1,f2,f3))//f1,f2,f3 in one row

import org.apache.spark.sql.types._
import org.apache.spark.sql.Row
f1: org.apache.spark.sql.types.StructField = StructField(name,StringType,false)
f2: org.apache.spark.sql.types.StructField = StructField(age,IntegerType,true)
f3: org.apache.spark.sql.types.StructField = StructField(income,DoubleType,true)
schema: org.apache.spark.sql.types.StructType = StructType(StructField(name,StringType,false),StructField(age,IntegerType,true),StructField(income,DoubleType,true))


<br><br>
<span style="color:blue;font-size:large">Make Row objects and construct the dataframe</span>
<p>
        <li>make sure that the data types are correct in the rowRDD. They have to match the schema!</li>

In [14]:
data

res11: Array[String] = Array(John,33,245233.234, Jill,23,132987.22, Qing,54,782344.22, Rahul,50,389223,54)


In [15]:
val rdd = sc.parallelize(data).map(r => r.split(","))
val rowRdd = rdd.map(a => Row(a(0),a(1).toInt,a(2).toDouble))
val df = spark.createDataFrame(rowRdd,schema)
df.printSchema

root
 |-- name: string (nullable = false)
 |-- age: integer (nullable = true)
 |-- income: double (nullable = true)



rdd: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[9] at map at <console>:33
rowRdd: org.apache.spark.rdd.RDD[org.apache.spark.sql.Row] = MapPartitionsRDD[10] at map at <console>:34
df: org.apache.spark.sql.DataFrame = [name: string, age: int ... 1 more field]


In [16]:
df.show

+-----+---+----------+
| name|age|    income|
+-----+---+----------+
| John| 33|245233.234|
| Jill| 23| 132987.22|
| Qing| 54| 782344.22|
|Rahul| 50|  389223.0|
+-----+---+----------+



In [17]:
rdd.collect

res14: Array[Array[String]] = Array(Array(John, 33, 245233.234), Array(Jill, 23, 132987.22), Array(Qing, 54, 782344.22), Array(Rahul, 50, 389223, 54))


In [18]:
rowRdd.collect

res15: Array[org.apache.spark.sql.Row] = Array([John,33,245233.234], [Jill,23,132987.22], [Qing,54,782344.22], [Rahul,50,389223.0])


<li>Use array style indexing to extract data from a Row RDD</li>
<li>Example: convert each Row object into a Scala List

In [19]:
rowRdd.map(t=>List(t(0),t(1))).collect

res16: Array[List[Any]] = Array(List(John, 33), List(Jill, 23), List(Qing, 54), List(Rahul, 50))


In [25]:
rowRdd.collect()
// (1)(1)

res22: Array[org.apache.spark.sql.Row] = Array([John,33,245233.234], [Jill,23,132987.22], [Qing,54,782344.22], [Rahul,50,389223.0])


<br><br>
<span style="color:green;font-size:xx-large">Working with dataframes</span>
<p>
        <li><span style="color:blue">df.schema</span> displays and returns the schema </li>
        <li><span style="color:blue">df.columns</span> returns the column names in an Array</li>
        <li><span style="color:blue">df.show</span> shows the dataframe in tabular format</li>
        <ul>
            <li>by default, show only displays 20 rows</li>
            <li><span style="color:blue">show with limit</span> controls the display length</li>            
            <li><span style="color:blue">take</span> also works, as does <span style="color:blue">first</span></li>

In [23]:
df.printSchema

root
 |-- name: string (nullable = false)
 |-- age: integer (nullable = true)
 |-- income: double (nullable = true)



In [24]:
val col_names = df.columns
col_names

col_names: Array[String] = Array(name, age, income)
res17: Array[String] = Array(name, age, income)


In [25]:
df.show

+-----+---+----------+
| name|age|    income|
+-----+---+----------+
| John| 33|245233.234|
| Jill| 23| 132987.22|
| Qing| 54| 782344.22|
|Rahul| 50|  389223.0|
+-----+---+----------+



<br><br>
<span style="color:blue;font-size:large">Seeing only a few rows</span>
<li>use <span style="color:blue">limit</span> to see a few rows</li>
<li>you can also use <span style="color:blue">take</span> but that will use rdd style display (no structure information)</li>

In [25]:
df.limit(2).show//default show 20 lines

+----+---+----------+
|name|age|    income|
+----+---+----------+
|John| 33|245233.234|
|Jill| 23| 132987.22|
+----+---+----------+



In [27]:
df.take(4)

res20: Array[org.apache.spark.sql.Row] = Array([John,33,245233.234], [Jill,23,132987.22], [Qing,54,782344.22], [Rahul,50,389223.0])


<br><br>
<span style="color:green;font-size:xx-large">Dataframe operations</span>
<li><span style="color:blue">select</span> chooses data from columns and can run grouped and aggregate operations on the dataframe</li>
<li>select returns a dataframe</li>
<li>select is a transformation and not an action</li>
<li>A \$ sign in front of a column name lets you use the column values like a variable</li>
<li><span style="color:blue">groupby</span> groups data on a column and can be used to get summary stats</li>
<li><span style="color:blue">filter</span> selects rows from a dataframe based on a condition</li>

In [26]:
val x = Array(("John","Q1",10),
              ("Jill","Q1",8),
              ("John","Q2",3),
              ("Jill","Q2",9))
val rdd = sc.parallelize(x)
val df = rdd.toDF("Name","Quiz","Score")
val names = df.select("Name")
names.show
val scores = df.select("Name","Score")
scores.show
val percentages = df.select($"Name",$"Score"*100.0/10.0)//$ -> trade "Score" as column. when you need one, you need to do every one. 
percentages.show
val percentages2 = df.select($"Name",$"Score"*100.0/10.0 as "percentage")
percentages2.show
val grouped = df.groupBy("Name")
grouped.mean("Score").show
val a_grades = df.filter($"Score">=9.0)
a_grades.show


+----+
|Name|
+----+
|John|
|Jill|
|John|
|Jill|
+----+

+----+-----+
|Name|Score|
+----+-----+
|John|   10|
|Jill|    8|
|John|    3|
|Jill|    9|
+----+-----+

+----+------------------------+
|Name|((Score * 100.0) / 10.0)|
+----+------------------------+
|John|                   100.0|
|Jill|                    80.0|
|John|                    30.0|
|Jill|                    90.0|
+----+------------------------+

+----+----------+
|Name|percentage|
+----+----------+
|John|     100.0|
|Jill|      80.0|
|John|      30.0|
|Jill|      90.0|
+----+----------+

+----+----------+
|Name|avg(Score)|
+----+----------+
|John|       6.5|
|Jill|       8.5|
+----+----------+

+----+----+-----+
|Name|Quiz|Score|
+----+----+-----+
|John|  Q1|   10|
|Jill|  Q2|    9|
+----+----+-----+



x: Array[(String, String, Int)] = Array((John,Q1,10), (Jill,Q1,8), (John,Q2,3), (Jill,Q2,9))
rdd: org.apache.spark.rdd.RDD[(String, String, Int)] = ParallelCollectionRDD[28] at parallelize at <console>:35
df: org.apache.spark.sql.DataFrame = [Name: string, Quiz: string ... 1 more field]
names: org.apache.spark.sql.DataFrame = [Name: string]
scores: org.apache.spark.sql.DataFrame = [Name: string, Score: int]
percentages: org.apache.spark.sql.DataFrame = [Name: string, ((Score * 100.0) / 10.0): double]
percentages2: org.apache.spark.sql.DataFrame = [Name: string, percentage: double]
grouped: org.apache.spark.sql.RelationalGroupedDataset = RelationalGroupedDataset: [grouping expressions: [Name: string], value: [Name: string, Quiz: string ... 1 more field], type: ]
a_grades: org.apache.spar...


<br><br>
<span style="color:blue;font-size:large">Working with columns</span>
<li>Adding a $ in front of a column name enables vector operations on a column</li>
<li>The <span style="color:blue">col</span> function can also be used to specify a column</li>
<li>two functions <span style="color:blue">withColumn</span> and <span style="color:blue">withColumnRenamed</span> add flexibility to columnar operations</li>
<li>withColumn takes two arguments, the name for the new column being created and an operation on a column, and returns a new dataframe</li>
<li>withColumnRenamed takes two arguments, the name of an existing column and the new name for that column and returns a new data frame</li>
<li>conditional expressions on columns use <span style="color:blue">when</span> and <span style="color:blue">otherwise</span>

In [27]:
df.printSchema

root
 |-- Name: string (nullable = true)
 |-- Quiz: string (nullable = true)
 |-- Score: integer (nullable = false)



In [28]:
val percentage = df.select($"Name",$"Score"*100.0/10.0)
percentage.show

+----+------------------------+
|Name|((Score * 100.0) / 10.0)|
+----+------------------------+
|John|                   100.0|
|Jill|                    80.0|
|John|                    30.0|
|Jill|                    90.0|
+----+------------------------+



percentage: org.apache.spark.sql.DataFrame = [Name: string, ((Score * 100.0) / 10.0): double]


In [32]:
val percentage = df.select($"Name",col("Score")*100.0/10.0)
percentage.show

+----+------------------------+
|Name|((Score * 100.0) / 10.0)|
+----+------------------------+
|John|                   100.0|
|Jill|                    80.0|
|John|                    30.0|
|Jill|                    90.0|
+----+------------------------+



percentage: org.apache.spark.sql.DataFrame = [Name: string, ((Score * 100.0) / 10.0): double]


In [32]:
//percentage is a new column in the df
val percentage = df.withColumn("percentage",col("Score")*100.0/10)
percentage.show

+----+----+-----+----------+
|Name|Quiz|Score|percentage|
+----+----+-----+----------+
|John|  Q1|   10|     100.0|
|Jill|  Q1|    8|      80.0|
|John|  Q2|    3|      30.0|
|Jill|  Q2|    9|      90.0|
+----+----+-----+----------+



percentage: org.apache.spark.sql.DataFrame = [Name: string, Quiz: string ... 2 more fields]


In [33]:
//finals is a df with column percentage renamed as final score
val finals = percentage.withColumnRenamed("percentage","final score")
finals.show


+----+----+-----+-----------+
|Name|Quiz|Score|final score|
+----+----+-----+-----------+
|John|  Q1|   10|      100.0|
|Jill|  Q1|    8|       80.0|
|John|  Q2|    3|       30.0|
|Jill|  Q2|    9|       90.0|
+----+----+-----+-----------+



finals: org.apache.spark.sql.DataFrame = [Name: string, Quiz: string ... 2 more fields]


<br><br>
<span style="color:blue;font-size:large">Conditional column values</span>
<li><span style="color:red">when</span> generates new column values conditionally</li>
<li>combine with <span style="color:red">withColumn</span> to create a new column</li>

In [35]:
val grades = percentage.withColumn("grade",
                      when(col("percentage")>=90.0,"A")
                      .when(col("percentage")>=80.0,"B")
                        .otherwise( "F"))
grades.show

+----+----+-----+----------+-----+
|Name|Quiz|Score|percentage|grade|
+----+----+-----+----------+-----+
|John|  Q1|   10|     100.0|    A|
|Jill|  Q1|    8|      80.0|    B|
|John|  Q2|    3|      30.0|    F|
|Jill|  Q2|    9|      90.0|    A|
+----+----+-----+----------+-----+



grades: org.apache.spark.sql.DataFrame = [Name: string, Quiz: string ... 3 more fields]


<br><br>
<span style="color:blue;font-size:large">Accessing nested columns</span>

In [36]:
df.show

+----+----+-----+
|Name|Quiz|Score|
+----+----+-----+
|John|  Q1|   10|
|Jill|  Q1|    8|
|John|  Q2|    3|
|Jill|  Q2|    9|
+----+----+-----+



In [37]:
val df = spark.read.json("employees_singleline.json")
df.printSchema

root
 |-- name: string (nullable = true)
 |-- positions: struct (nullable = true)
 |    |-- NYC: long (nullable = true)
 |    |-- Palo Alto: long (nullable = true)
 |-- salary: long (nullable = true)



df: org.apache.spark.sql.DataFrame = [name: string, positions: struct<NYC: bigint, Palo Alto: bigint> ... 1 more field]


In [38]:
df.show

+-------+---------+------+
|   name|positions|salary|
+-------+---------+------+
|Michael|   {1, 2}|  3000|
|   Andy|{1, null}|  4500|
| Justin|   {1, 2}|  3500|
|  Berta|   {1, 2}|  4000|
+-------+---------+------+



In [39]:
df.select("positions").show

+---------+
|positions|
+---------+
|   {1, 2}|
|{1, null}|
|   {1, 2}|
|   {1, 2}|
+---------+



In [40]:
df.select("name","positions.NYC").show

+-------+---+
|   name|NYC|
+-------+---+
|Michael|  1|
|   Andy|  1|
| Justin|  1|
|  Berta|  1|
+-------+---+



<br><br>
<span style="color:green;font-size:xx-large">User-defined functions</span>
<p>
        <li>It is possible to define a function that can be applied to each element in a column (or columns)</li>
        <li>Define a function that works on one element</li>
    <li>add it to the <span style="color:red">udf</span> library</li>
        <li>apply it to a column (or columns)</li>

In [41]:
val x = Array(("John","Q1",10),
              ("Jill","Q1",8),
              ("John","Q2",3),
              ("Jill","Q2",9))
val rdd = sc.parallelize(x)
val df = rdd.toDF("Name","Quiz","Score")
df.show

+----+----+-----+
|Name|Quiz|Score|
+----+----+-----+
|John|  Q1|   10|
|Jill|  Q1|    8|
|John|  Q2|    3|
|Jill|  Q2|    9|
+----+----+-----+



x: Array[(String, String, Int)] = Array((John,Q1,10), (Jill,Q1,8), (John,Q2,3), (Jill,Q2,9))
rdd: org.apache.spark.rdd.RDD[(String, String, Int)] = ParallelCollectionRDD[92] at parallelize at <console>:36
df: org.apache.spark.sql.DataFrame = [Name: string, Quiz: string ... 1 more field]


In [42]:
def grader(x: Int): String = 
    if (x >= 9) "A"
    else if (x >= 8) "B"
    else "F"

println(grader(9))
println(grader(3))
println(grader(8))

A
F
B


grader: (x: Int)String


In [43]:
import org.apache.spark.sql.functions.udf
val grader_udf = udf(grader _) //defines a udf as "Call grader with arguments" for cols
val grades_df = df.withColumn("grade",grader_udf($"Score"))
grades_df.show

+----+----+-----+-----+
|Name|Quiz|Score|grade|
+----+----+-----+-----+
|John|  Q1|   10|    A|
|Jill|  Q1|    8|    B|
|John|  Q2|    3|    F|
|Jill|  Q2|    9|    A|
+----+----+-----+-----+



import org.apache.spark.sql.functions.udf
grader_udf: org.apache.spark.sql.expressions.UserDefinedFunction = SparkUserDefinedFunction($Lambda$4966/0x0000000801a29040@2400752b,StringType,List(Some(class[value[0]: int])),Some(class[value[0]: string]),None,true,true)
grades_df: org.apache.spark.sql.DataFrame = [Name: string, Quiz: string ... 2 more fields]


<br><br>
<span style="color:green;font-size:xx-large">Aggregate functions</span>
<p>
<li><a href="https://spark.apache.org/docs/2.4.7/api/scala/index.html#org.apache.spark.sql.functions$">documentation</a></li>

In [76]:
df.agg(max("Score"),stddev("Score")).show

+----------+------------------+
|max(Score)|stddev_samp(Score)|
+----------+------------------+
|        10| 2.701851217221259|
+----------+------------------+



In [44]:
df.agg(max("Score")).show
df.agg(stddev("Score")).show
df.agg(avg("Score")).show

+----------+
|max(Score)|
+----------+
|        10|
+----------+

+------------------+
|stddev_samp(Score)|
+------------------+
|3.1091263510296048|
+------------------+

+----------+
|avg(Score)|
+----------+
|       7.5|
+----------+



<br><br>
<span style="color:blue;font-size:x-large">User defined aggregate functions</span>
        <li>Example: compute the mean processing time for cases that take more than one day</li>

In [52]:
val df = spark.read
    .format("csv")
    .option("header","true")
    .option("inferSchema","true")
   .csv("nyc_311_2022_clean.csv")
    .withColumnRenamed("Created Date","Created_Date")
    .withColumnRenamed("Closed Date","Closed_Date")

df: org.apache.spark.sql.DataFrame = [Created_Date: timestamp, Closed_Date: timestamp ... 9 more fields]


In [53]:
df.printSchema

root
 |-- Created_Date: timestamp (nullable = true)
 |-- Closed_Date: timestamp (nullable = true)
 |-- Agency: string (nullable = true)
 |-- Agency Name: string (nullable = true)
 |-- Complaint Type: string (nullable = true)
 |-- Incident Zip: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- processing_time: string (nullable = true)
 |-- processing_days: double (nullable = true)



In [54]:
//Necessary imports
import org.apache.spark.sql.{Row, SparkSession}
import org.apache.spark.sql.expressions.MutableAggregationBuffer
import org.apache.spark.sql.expressions.UserDefinedAggregateFunction
import org.apache.spark.sql.types._

//We'll define a scala program that calculates this average
//Scala programs are contained in an object
//object can only have one instance
//object, class, data, function are the same level
object ConditionalAverage extends UserDefinedAggregateFunction {
    //Start with a schema for the input to the function
    //In this case, a column and the type of the column
    def inputSchema: StructType = StructType(StructField("inputColumn", DoubleType) ::
                                             //StructField("inputColumn", StringType)
                                             Nil)

    // Set up an aggregation buffer. This specified the schema for the data that will be updated by each case
    // Since we're calculating an average, we need to count elements and sum elements
    def bufferSchema: StructType = {
        StructType(StructField("sum", DoubleType) :: StructField("count", LongType) :: Nil)
    }
    
    //Define the type of the value returned by the function
    def dataType: DataType = DoubleType
    
    // Whether this function always returns the same output on the identical input
    def deterministic: Boolean = true
    
    //Initialize the aggregation buffer
    def initialize(buffer: MutableAggregationBuffer): Unit = {
        buffer(0) = 0.0
        buffer(1) = 0L //0 long 
    }
    
    // Update procedure. What to do with each case
    //the buffer argument is the buffer above
    //the input argument is a single row. For each row, it extracts the data from the column or columns
    def update(buffer: MutableAggregationBuffer, input: Row): Unit = {
        if (!input.isNullAt(0)) { //0 is the location
            if (input.getDouble(0) > 1.0) {
                buffer(0) = buffer.getDouble(0) + input.getDouble(0) //The sum of the values
                buffer(1) = buffer.getLong(1) + 1 //The count of the values
            }
        }
    }
    
    // Merges two aggregation buffers and stores the updated buffer values back to `buffer1`
    // Because - partitions!
    // buffer1 is the final buffer
    def merge(buffer1: MutableAggregationBuffer, buffer2: Row): Unit = {
        buffer1(0) = buffer1.getDouble(0) + buffer2.getDouble(0)
        buffer1(1) = buffer1.getLong(1) + buffer2.getLong(1)
    }
    // Calculates the final result (the average)
    //the evaluate function converts the buffer into the output
    def evaluate(buffer: Row): Double = buffer.getDouble(0) / buffer.getLong(1)
}

// Register the function to access it
spark.udf.register("ConditionalAverage", ConditionalAverage) //first argument is name, second is function



22/11/06 15:33:04 WARN SimpleFunctionRegistry: The function conditionalaverage replaced a previously registered function.


import org.apache.spark.sql.{Row, SparkSession}
import org.apache.spark.sql.expressions.MutableAggregationBuffer
import org.apache.spark.sql.expressions.UserDefinedAggregateFunction
import org.apache.spark.sql.types._
defined object ConditionalAverage
res44: org.apache.spark.sql.expressions.UserDefinedAggregateFunction = ConditionalAverage$@313096c2


<b>object ConditionalAverage extends UserDefinedAggregateFunction</b>

* Since the function has several components, collect it inside an object (if unparameterized) or a class (if parameterized)
* And we extend the UserDefinedAggregateFunction class. Note that the buffers etc. defined below are attributes of this class

<b>def inputSchema: StructType = StructType(StructField("inputColumn", DoubleType) :: Nil)</b>

The input schema sets up the columns that the aggregate function will use. Multiple columns are permitted (constants use "lit") and the schema is organized as a sequence of StructFields. In the example above, the sequence has only one element and we're constructing the sequence by cons-ing that element with Nil. 

Types in StructField are obvious: DoubleType, LongType, IntType, BooleanType, StringType, etc. See <a href="https://spark.apache.org/docs/latest/sql-ref-datatypes.html">the documentation</a> for all types

<b>def bufferSchema: StructType = {
        StructType(StructField("sum", DoubleType) :: StructField("count", LongType) :: Nil)
    }</b>
    
An aggregator takes each row and does something with the values in that row. The result of that "something" is maintained in the buffer. The buffer is also a sequence of StructField like the input schema. In our example, we need to maintain the sum and the count as the function iterates through values 

<b>def dataType: DataType = DoubleType</b>

The data type of the "aggregated" value returned by the function

<b>def deterministic: Boolean = true</b>

If true, the function always return the same value for the same input. deterministic may be false if, for example, there is a lot of data and the aggregate function randomly samples some of the data rather than all of it



In [57]:
df.agg(ConditionalAverage(df("processing_days"))).show

+------------------------------------+
|conditionalaverage$(processing_days)|
+------------------------------------+
|                  19.811219247322303|
+------------------------------------+



In [53]:
// df.agg(avg("processing_days")).show

<b>Example 2</b>

<li>Suppose we want to be able to specify the threshold value (i.e., the 1 day in our example above)</li>
<li>We will add a second argument (using Cons) to the inputSchema</li>
<li>And use this value in the update procedure instead of 1.0</li>
<li>Since sparkSQL requires that all arguments to inputSchema be Column data types, we need to convert the threshold value into a column when calling the aggregate function. The <b>lit</b> function does this</li>

In [54]:
//Necessary imports
import org.apache.spark.sql.{Row, SparkSession}
import org.apache.spark.sql.expressions.MutableAggregationBuffer
import org.apache.spark.sql.expressions.UserDefinedAggregateFunction
import org.apache.spark.sql.types._

//We'll define a scala program that calculates this average
//Scala programs are contained in an object
object ConditionalAverage extends UserDefinedAggregateFunction {
    //Start with a schema for the input to the function
    //In this case, a column and the type of the column
    def inputSchema: StructType = StructType(StructField("inputColumn", DoubleType) :: 
                                             StructField("threshold",DoubleType) :: Nil)

    // Set up an aggregation buffer. This specified the schema for the data that will be updated by each case
    // Since we're calculating an average, we need to count elements and sum elements
    def bufferSchema: StructType = {
        StructType(StructField("sum", DoubleType) :: StructField("count", LongType) :: Nil)
    }
    
    //Define the type of the value returned by the function
    def dataType: DataType = DoubleType
    
    // Whether this function always returns the same output on the identical input
    def deterministic: Boolean = true
    
    //Initialize the aggregation buffer
    def initialize(buffer: MutableAggregationBuffer): Unit = {
        buffer(0) = 0.0
        buffer(1) = 0L
    }
    
    // Update procedure. What to do with each case
    def update(buffer: MutableAggregationBuffer, input: Row): Unit = {
        if (!input.isNullAt(0)) {
            if (input.getDouble(0) > input.getDouble(1)) {
                buffer(0) = buffer.getDouble(0) + input.getDouble(0)
                buffer(1) = buffer.getLong(1) + 1
            }
        }
    }
    
    // Merges two aggregation buffers and stores the updated buffer values back to `buffer1`
    def merge(buffer1: MutableAggregationBuffer, buffer2: Row): Unit = {
        buffer1(0) = buffer1.getDouble(0) + buffer2.getDouble(0)
        buffer1(1) = buffer1.getLong(1) + buffer2.getLong(1)
    }
  // Calculates the final result
    def evaluate(buffer: Row): Double = buffer.getDouble(0) / buffer.getLong(1)
}

// Register the function to access it
spark.udf.register("ConditionalAverage", ConditionalAverage)



22/10/26 10:31:49 WARN SimpleFunctionRegistry: The function conditionalaverage replaced a previously registered function.


import org.apache.spark.sql.{Row, SparkSession}
import org.apache.spark.sql.expressions.MutableAggregationBuffer
import org.apache.spark.sql.expressions.UserDefinedAggregateFunction
import org.apache.spark.sql.types._
defined object ConditionalAverage
res45: org.apache.spark.sql.expressions.UserDefinedAggregateFunction = ConditionalAverage$@1d8929ab


In [55]:
df.agg(ConditionalAverage(df("processing_days"),lit(1.0))).show

+-----------------------------------------+
|conditionalaverage$(processing_days, 1.0)|
+-----------------------------------------+
|                       19.811219247322303|
+-----------------------------------------+



<h4>Question</h4>
What modifications would we need to do to ConditionalAverage if we wanted to:
<li>use an interval (e.g., average of times between 1 and 5 days)</li>
<li>include something that lets us flip between greater and lesser</li>

In [56]:
lit(1.0)

res47: org.apache.spark.sql.Column = 1.0


In [57]:
df("processing_days")

res48: org.apache.spark.sql.Column = processing_days


In [60]:
df.select(col("processing_days"))

res51: org.apache.spark.sql.DataFrame = [processing_days: double]


<br><br>
<span style="color:green;font-size:xx-large">Running SQL queries</span>
<p>
<li>Since a dataframe is like an SQL table (like-ish)</li>
<li> We can run SQL queries on a df</li>
<li>Set up a temporary view or a global view</li>
<li>Send SQL queries to the dataframe</li>


In [58]:
!head nyc_311_2022_clean.csv


Created Date,Closed Date,Agency,Agency Name,Complaint Type,Incident Zip,Borough,Latitude,Longitude,processing_time,processing_days
2020-01-07 14:09:00,2020-01-13 11:20:00,DSNY,Department of Sanitation,Electronics Waste Appointment,11692,QUEENS,40.58993519447414,-73.78942049765358,5 days 21:11:00,5.882638888888889
2020-01-04 11:37:00,2020-01-08 13:19:00,DSNY,Department of Sanitation,Electronics Waste Appointment,10310,STATEN ISLAND,40.62719924888892,-74.11245623591475,4 days 01:42:00,4.070833333333334
2020-01-03 16:33:00,2020-01-05 00:00:00,DSNY,Department of Sanitation,Request Large Bulky Item Collection,11213,BROOKLYN,40.6672052181697,-73.93463635283278,1 days 07:27:00,1.3104166666666668
2020-01-06 17:27:00,2020-01-11 00:00:00,DSNY,Department of Sanitation,Request Large Bulky Item Collection,11379,QUEENS,40.72687041685842,-73.8769198480706,4 days 06:33:00,4.272916666666666
2020-01-06 09:46:00,2020-01-08 00:00:00,DSNY,Department of Sanitation,Request Large Bulky Item Collection,11221,B

In [59]:
val df = spark.read
    .format("csv")
    .option("header","true")
    //.option("inferSchema","true")
   .csv("nyc_311_2022_clean.csv")
    .withColumnRenamed("Created Date","Created_Date")
    .withColumnRenamed("Closed Date","Closed_Date")




df: org.apache.spark.sql.DataFrame = [Created_Date: string, Closed_Date: string ... 9 more fields]


In [60]:
df.printSchema

root
 |-- Created_Date: string (nullable = true)
 |-- Closed_Date: string (nullable = true)
 |-- Agency: string (nullable = true)
 |-- Agency Name: string (nullable = true)
 |-- Complaint Type: string (nullable = true)
 |-- Incident Zip: string (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- processing_time: string (nullable = true)
 |-- processing_days: string (nullable = true)



In [61]:
df.take(5).foreach(println)

[2020-01-07 14:09:00,2020-01-13 11:20:00,DSNY,Department of Sanitation,Electronics Waste Appointment,11692,QUEENS,40.58993519447414,-73.78942049765358,5 days 21:11:00,5.882638888888889]
[2020-01-04 11:37:00,2020-01-08 13:19:00,DSNY,Department of Sanitation,Electronics Waste Appointment,10310,STATEN ISLAND,40.62719924888892,-74.11245623591475,4 days 01:42:00,4.070833333333334]
[2020-01-03 16:33:00,2020-01-05 00:00:00,DSNY,Department of Sanitation,Request Large Bulky Item Collection,11213,BROOKLYN,40.6672052181697,-73.93463635283278,1 days 07:27:00,1.3104166666666668]
[2020-01-06 17:27:00,2020-01-11 00:00:00,DSNY,Department of Sanitation,Request Large Bulky Item Collection,11379,QUEENS,40.72687041685842,-73.8769198480706,4 days 06:33:00,4.272916666666666]
[2020-01-06 09:46:00,2020-01-08 00:00:00,DSNY,Department of Sanitation,Request Large Bulky Item Collection,11221,BROOKLYN,40.68911879550976,-73.93833612355152,1 days 14:14:00,1.5930555555555554]


<br><br>
<span style="color:blue;font-size:x-large">SQL table and queries</span>
        <li>Create an SQL table</li>
        <li>And then query it using SQL syntax</li>

In [62]:
df.printSchema

root
 |-- Created_Date: string (nullable = true)
 |-- Closed_Date: string (nullable = true)
 |-- Agency: string (nullable = true)
 |-- Agency Name: string (nullable = true)
 |-- Complaint Type: string (nullable = true)
 |-- Incident Zip: string (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- processing_time: string (nullable = true)
 |-- processing_days: string (nullable = true)



In [63]:
df.createOrReplaceTempView("complaints") //Creates a temporary SQL-like table
val time_df = spark.sql("select agency, processing_time from complaints")
val nypd_high = spark.sql("select * FROM complaints WHERE agency='NYPD' AND processing_days>0.5")

time_df: org.apache.spark.sql.DataFrame = [agency: string, processing_time: string]
nypd_high: org.apache.spark.sql.DataFrame = [Created_Date: string, Closed_Date: string ... 9 more fields]


In [78]:
time_df.limit(4).show
nypd_high.limit(4).show

+------+---------------+
|agency|processing_time|
+------+---------------+
|  DSNY|5 days 21:11:00|
|  DSNY|4 days 01:42:00|
|  DSNY|1 days 07:27:00|
|  DSNY|4 days 06:33:00|
+------+---------------+

+-------------------+-------------------+------+--------------------+-------------------+------------+-------+------------------+------------------+---------------+------------------+
|       Created_Date|        Closed_Date|Agency|         Agency Name|     Complaint Type|Incident Zip|Borough|          Latitude|         Longitude|processing_time|   processing_days|
+-------------------+-------------------+------+--------------------+-------------------+------------+-------+------------------+------------------+---------------+------------------+
|2019-07-18 01:27:44|2019-07-18 16:18:24|  NYPD|New York City Pol...|Noise - Residential|       10466|  BRONX| 40.88797508393539|-73.85580773230332|0 days 14:50:40|0.6185185185185185|
|2020-03-15 19:51:35|2020-03-17 02:09:05|  NYPD|New York City P

<br><br>
<span style="color:blue;font-size:x-large">Using variables inside an SQL query</span>
<li>Because SQL query values need to be quoted inside the query string, variables require special care</li>
<li>

In [65]:
//This won't work because ag will be replaced by the unquoted string NYPD
//Spark will look for a column named NYPD
val ag="NYPD"
spark.sql(s"select * from complaints where agency=$ag and processing_time>1.0").show

org.apache.spark.sql.AnalysisException:  Column 'ag' does not exist. Did you mean one of the following? [complaints.Agency, complaints.Borough, complaints.Latitude, complaints.Longitude, complaints.Agency Name, complaints.Closed_Date, complaints.Created_Date, complaints.Incident Zip, complaints.Complaint Type, complaints.processing_days, complaints.processing_time]; line 1 pos 38;

In [80]:
val x = """NYPD"""
x

x: String = NYPD
res64: String = NYPD


In [81]:
val ag=""""NYPD""""
ag

ag: String = "NYPD"
res65: String = "NYPD"


In [68]:
//This will work. The three quotes will set the value of ag to NYPD with two quotes (six characters)
val ag=""""DOE""""
spark.sql(s"select * from complaints where agency=$ag and processing_days>0.5").show

+-------------------+-------------------+------+--------------------+------------------+------------+---------+------------------+------------------+----------------+------------------+
|       Created_Date|        Closed_Date|Agency|         Agency Name|    Complaint Type|Incident Zip|  Borough|          Latitude|         Longitude| processing_time|   processing_days|
+-------------------+-------------------+------+--------------------+------------------+------------+---------+------------------+------------------+----------------+------------------+
|2020-07-23 13:08:44|2020-08-25 14:56:03|   DOE|Department of Edu...|School Maintenance|       10467|    BRONX| 40.88562974190851|-73.86605482246202|33 days 01:47:19| 33.07452546296296|
|2020-03-13 15:43:56|2020-06-19 15:39:18|   DOE|Department of Edu...|School Maintenance|       10031|MANHATTAN| 40.83050367593391|-73.94449624244284|97 days 23:55:22| 97.99678240740741|
|2020-03-10 17:27:21|2020-06-03 08:52:47|   DOE|Department of Edu...|S

ag: String = "DOE"


<br><br>
<span style="color:blue;font-size:x-large">Calculating averages by agency</span>
<li>We used combineByKey when calculating this on RDD datasets</li>
<li>Life is a lot simpler with SQL!</li>

In [85]:
df.printSchema

root
 |-- Created_Date: string (nullable = true)
 |-- Closed_Date: string (nullable = true)
 |-- Agency: string (nullable = true)
 |-- Agency Name: string (nullable = true)
 |-- Complaint Type: string (nullable = true)
 |-- Incident Zip: string (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- processing_time: string (nullable = true)
 |-- processing_days: string (nullable = true)



In [86]:
val agency_averages = spark.sql("select Agency, AVG(processing_days) from complaints GROUP BY Agency")
agency_averages.show

+--------------------+--------------------+
|              Agency|avg(processing_days)|
+--------------------+--------------------+
|MAYORâS OFFICE ...|  10.993121710572932|
|                 DPR|   65.99335925229708|
|                 TLC|   53.68203580195775|
|                 DOE|   43.44743418663761|
|                 DOB|  39.106442592865434|
|               DOHMH|  15.397117888616267|
|                 DEP|   5.006676030990062|
|                 DOT|  14.494687185115172|
|                 DOF|   19.76244362870433|
|                DSNY|   6.984900241530156|
|                 DCA|  3.1134426905912487|
|                 EDC|   56.00096076391825|
|                NYPD|  0.3395727580073328|
|               DOITT|  28.392176800309258|
|                 HPD|  13.242378653308977|
|                 DHS|   1.257410746037668|
|                DFTA|  13.390725308641976|
|OFFICE OF TECHNOL...|  0.7780439814814815|
|                 OSE| 0.12648533950617283|
|                FDNY|   402.144

agency_averages: org.apache.spark.sql.DataFrame = [Agency: string, avg(processing_days): double]


<br><br>
<span style="color:blue;font-size:x-large">SQL joins</span>


In [87]:
import spark.implicits._ //This will let us convert a Scala Seq of arbitrary objects to a df
case class Student(id_no: String,name: String, age: Int)
case class EnrolledIn(id_no: String,course_no: String,course_name: String)

    
val student_df = Seq(Student("S1","Jack",30),Student("S2","Jill",20)).toDF
val enrolled_df = Seq(EnrolledIn("S1","C1","Cloud"),EnrolledIn("S2","C1","Cloud"),EnrolledIn("S1","C2","ML")).toDF

student_df.createOrReplaceTempView("students")
enrolled_df.createOrReplaceTempView("enrolled")

import spark.implicits._
defined class Student
defined class EnrolledIn
student_df: org.apache.spark.sql.DataFrame = [id_no: string, name: string ... 1 more field]
enrolled_df: org.apache.spark.sql.DataFrame = [id_no: string, course_no: string ... 1 more field]


In [88]:
student_df.show

+-----+----+---+
|id_no|name|age|
+-----+----+---+
|   S1|Jack| 30|
|   S2|Jill| 20|
+-----+----+---+



In [89]:
enrolled_df.show

+-----+---------+-----------+
|id_no|course_no|course_name|
+-----+---------+-----------+
|   S1|       C1|      Cloud|
|   S2|       C1|      Cloud|
|   S1|       C2|         ML|
+-----+---------+-----------+



<br><br>
<span style="color:blue;font-size:x-large">writing an inner join</span>
<li>get all the courses that Jack is currently registered in</li>

In [90]:
student_df.show
enrolled_df.show

+-----+----+---+
|id_no|name|age|
+-----+----+---+
|   S1|Jack| 30|
|   S2|Jill| 20|
+-----+----+---+

+-----+---------+-----------+
|id_no|course_no|course_name|
+-----+---------+-----------+
|   S1|       C1|      Cloud|
|   S2|       C1|      Cloud|
|   S1|       C2|         ML|
+-----+---------+-----------+



In [92]:
val c = """"Jack""""
spark.sql(s"select students.name, enrolled.course_name from students inner join enrolled on students.id_no = enrolled.id_no where students.name=$c").show

+----+-----------+
|name|course_name|
+----+-----------+
|Jack|      Cloud|
|Jack|         ML|
+----+-----------+



c: String = "Jack"


<h1>DataFrames are functional objects</h1>
<li>Which means we can chain function calls</li>

In [111]:
def categorizer(x: Double): Int = 
    if (x < 1.0) 0
    else if (x < 5.0) 1
    else 2
val categorizer_udf = udf(categorizer _) //defines a udf as "Call categorizer with an argument"


df.withColumn("proc_time_category",categorizer_udf($"processing_days"))
    .filter($"proc_time_category"==="1")
    .groupBy("Agency")
    .agg(ConditionalAverage(col("processing_days"),lit(1.0)))
    .withColumnRenamed("conditionalaverage$(processing_days, 1.0)","mean_proc_time")
    .select("Agency","mean_proc_time")
    .show

+--------------------+------------------+
|              Agency|    mean_proc_time|
+--------------------+------------------+
|MAYORâS OFFICE ...|2.4033759376747463|
|                 DPR|2.7649726904896768|
|                 TLC|2.3661278584455667|
|                 DOB|2.5542114746715168|
|               DOHMH|  2.10285441492303|
|                 DEP| 2.385515127231277|
|                 DOT|2.5067696788362825|
|                 DOF|2.7180876825601366|
|                DSNY| 2.451787242140279|
|                 DCA|2.1624431795090624|
|                NYPD| 2.033820700596487|
|                 HPD|2.7037918624513955|
|                 DHS|2.1804076945162225|
|                 DOE| 2.574423699906918|
|               DOITT|2.4433302696078436|
|                 EDC| 2.939592983254035|
+--------------------+------------------+



categorizer: (x: Double)Int
categorizer_udf: org.apache.spark.sql.expressions.UserDefinedFunction = SparkUserDefinedFunction($Lambda$5948/0x0000000801d6e040@6f29f505,IntegerType,List(Some(class[value[0]: double])),Some(class[value[0]: int]),None,false,true)


<h2>Dataframes and datasets can handle missing values</h2>

In [73]:

val y = sc.parallelize(Array(("John","56","New York"),("Jill","80","Boston"),
              ("Jacoby","","Phoenix"),("Jabberwocky","17","New York")))
val y2 = y.map(t => {
    try {
        (t._1,Some(t._2.toInt),t._3)
    } catch {
        case e: Exception => (t._1,None,t._3)
    }
})
val ds = y2.toDF("name","age","city")
ds.show
ds.select(avg("age")).show

+-----------+----+--------+
|       name| age|    city|
+-----------+----+--------+
|       John|  56|New York|
|       Jill|  80|  Boston|
|     Jacoby|null| Phoenix|
|Jabberwocky|  17|New York|
+-----------+----+--------+

+--------+
|avg(age)|
+--------+
|    51.0|
+--------+



y: org.apache.spark.rdd.RDD[(String, String, String)] = ParallelCollectionRDD[149] at parallelize at <console>:52
y2: org.apache.spark.rdd.RDD[(String, Option[Int], String)] = MapPartitionsRDD[150] at map at <console>:54
ds: org.apache.spark.sql.DataFrame = [name: string, age: int ... 1 more field]


In [27]:
(56+80+17)/3

res24: Int = 51


In [70]:
ds.agg(avg("age")).show

+--------+
|avg(age)|
+--------+
|    51.0|
+--------+



<br><br>
<span style="color:green;font-size:xx-large">Dataset</span>
<br>
<li>Sort of intermediate between an RDD and a DF</li>
<li>They are type safe unlike dataframes</li>
<li>Can have a schema (like dataframes)</li>
<li>Are performance optimized (like dataframes)</li>
<li>Python and R Spark APIs do not support datasets</li>


In [74]:
import spark.implicits._

val x = Array(("John","Q1",10),
              ("Jill","Q1",8),
              ("John","Q2",3),
              ("Jill","Q2",9),
            ("Bill","Q2",7))
val rdd = sc.parallelize(x)
val df = rdd.toDF("Name","Quiz","Score")

case class grades(name: String,quiz: String, score: Int)
val ds_from_df = df.as[grades] //Creates a dataset from a DataFrame and uses df schema
val ds_from_rdd = rdd.toDS //Creates a dataset from an RDD with a default schema

import spark.implicits._
x: Array[(String, String, Int)] = Array((John,Q1,10), (Jill,Q1,8), (John,Q2,3), (Jill,Q2,9), (Bill,Q2,7))
rdd: org.apache.spark.rdd.RDD[(String, String, Int)] = ParallelCollectionRDD[160] at parallelize at <console>:59
df: org.apache.spark.sql.DataFrame = [Name: string, Quiz: string ... 1 more field]
defined class grades
ds_from_df: org.apache.spark.sql.Dataset[grades] = [Name: string, Quiz: string ... 1 more field]
ds_from_rdd: org.apache.spark.sql.Dataset[(String, String, Int)] = [_1: string, _2: string ... 1 more field]


In [96]:
rdd.collect

res78: Array[(String, String, Int)] = Array((John,Q1,10), (Jill,Q1,8), (John,Q2,3), (Jill,Q2,9), (Bill,Q2,7))


In [97]:
df.printSchema

root
 |-- Name: string (nullable = true)
 |-- Quiz: string (nullable = true)
 |-- Score: integer (nullable = false)



In [98]:
ds_from_df.printSchema

root
 |-- Name: string (nullable = true)
 |-- Quiz: string (nullable = true)
 |-- Score: integer (nullable = false)



In [99]:
ds_from_rdd.printSchema

root
 |-- _1: string (nullable = true)
 |-- _2: string (nullable = true)
 |-- _3: integer (nullable = false)



In [100]:
ds_from_df.collect

res82: Array[grades] = Array(grades(John,Q1,10), grades(Jill,Q1,8), grades(John,Q2,3), grades(Jill,Q2,9), grades(Bill,Q2,7))


In [101]:
ds_from_df.select("Name").show
ds_from_rdd.select("_1").show

+----+
|Name|
+----+
|John|
|Jill|
|John|
|Jill|
|Bill|
+----+

+----+
|  _1|
+----+
|John|
|Jill|
|John|
|Jill|
|Bill|
+----+



<br><br>
<span style="color:green;font-size:xx-large">Dataframes, RDDs, Datasets</span>
<li>RDD</li>
<ul>
    <li>Data is unstructured (no schema)</li>
    <li>Useful when you want to do low level transformations</li>
    <li>More control over the data (you can impose any schema-in-the-mind on the data)</li>
    <li>Schema does not matter to you (because you're good at functional programming!)</li>
    <li>RDDs are type safe</li>
</ul>
<li>Dataframes</li>
<ul>
    <li>Structured (RDBMS) or semi-structured (JSON/CSV) data</li>
    <li>Uses an SQL optimized engine that converts operations into low level RDD transformations (efficient and fast)</li>
    <li>Better for PySpark or other non-functional programming environment</li>
    <li>Better if you like SQL over functional languages</li>
    <li>Dataframes are not type safe (a non-existent column will not be detected until it is referenced</li>
</ul>
<li>Datasets</li>
<ul>
    <li>Not available for pyspark</li>
    <li>Sort of intermediate between an RDD and a Dataframe</li>
    <li>More efficient than an RDD but less than a dataframe</li>
    <li>Datasets are type safe</li>
</ul>